In [1]:
%reload_ext autoreload
%autoreload 

In [3]:
from mongodb import MongoDB
from db_utils import MongoUtils


ModuleNotFoundError: No module named 'chatbot'

In [12]:
## Connect to mongo client
mongo_username ="csc648"
mongo_pass = "6So9gzE3Jw5Oenup"

mongo_client = MongoDB(mongo_username, mongo_pass).connect()

2024-10-23 17:38:14.285 | DEBUG    | mongodb:connect:30 - Successfully connected to MongoDB!


In [13]:
# Update embeddings

db_name = "chillmate"
collection_name = "Resources"

mongoutils = MongoUtils(mongo_client, db_name, collection_name)
mongoutils.update_embeddings()


2024-10-23 17:38:24.837 | DEBUG    | db_utils:update_embeddings:41 - Updated document 67158ff557acd5513da9bad3 with embeddings.
2024-10-23 17:38:25.183 | DEBUG    | db_utils:update_embeddings:41 - Updated document 6715915057acd5513da9bad5 with embeddings.
2024-10-23 17:38:25.501 | DEBUG    | db_utils:update_embeddings:41 - Updated document 671591b057acd5513da9bad7 with embeddings.
2024-10-23 17:38:25.812 | DEBUG    | db_utils:update_embeddings:41 - Updated document 6715920c57acd5513da9bad8 with embeddings.
2024-10-23 17:38:26.115 | DEBUG    | db_utils:update_embeddings:41 - Updated document 67194ff8c954edc179dc8056 with embeddings.
2024-10-23 17:38:26.388 | DEBUG    | db_utils:update_embeddings:41 - Updated document 67195051c954edc179dc8057 with embeddings.
2024-10-23 17:38:26.658 | DEBUG    | db_utils:update_embeddings:41 - Updated document 6719509cc954edc179dc8059 with embeddings.
2024-10-23 17:38:26.923 | DEBUG    | db_utils:update_embeddings:41 - Updated document 671950e8c954edc179

In [14]:
# Find most similar document to input query
query = """Free Food on campus"""

# Get vector embeddings for the input query
input_vec_embeddings = mongoutils.generate_embeddings(query)


In [15]:
input_vec_embeddings

[0.05500442,
 -0.020895312,
 -0.0037109705,
 -0.06755454,
 0.0033432124,
 0.0429199,
 0.0072240476,
 -0.049790535,
 0.019488549,
 0.05631346,
 -0.0022305043,
 1.0971859e-05,
 -0.022023058,
 -0.027327893,
 -0.039208107,
 -0.007969772,
 0.0069623664,
 0.050016318,
 0.038942456,
 -0.065947235,
 0.02168802,
 0.0141705405,
 -0.006914428,
 -0.02817802,
 0.009357894,
 0.039659195,
 0.05862347,
 -0.040398248,
 -0.014034709,
 0.017546616,
 -0.07371733,
 0.029441977,
 -0.054484326,
 0.0039356416,
 0.006798354,
 -0.039925817,
 0.0033362822,
 0.026132377,
 0.037920956,
 0.077700935,
 -0.0056491513,
 0.027496131,
 -0.063366115,
 -0.047457375,
 0.006264708,
 0.007606025,
 -0.015702594,
 0.022258714,
 0.030404102,
 -0.07815185,
 -0.0420274,
 0.007842178,
 0.024176272,
 0.019323677,
 -0.036294214,
 -0.0130297085,
 0.038071647,
 0.0011353508,
 -0.0077351113,
 -0.030408101,
 0.024986383,
 0.017844586,
 0.018509809,
 0.021981707,
 -0.032519285,
 -0.04416789,
 -0.029283823,
 0.02630241,
 0.048953567,
 0.0

In [16]:
similar_docs = mongoutils.find_similar_documents(inp_document_embedding=input_vec_embeddings, embedding_name="ResourceEmbedding", no_of_docs=2)

In [17]:
for doc, score in similar_docs:
    resource_title = doc.get('RecourseTitle')
    resource_link = doc.get("RecourseLink")
    resource_body = doc.get('ResourseBody')

    print(f'------')
    print(f"Resource Title: {resource_title}")
    print(f"Resource Link: {resource_link}")
    print(f"Resource Body: {resource_body}")
    print(f"Cosine similarity score: {score}")
    print(f'------')

------
Resource Title: Basic Needs
Resource Link: https://basicneeds.sfsu.edu/
Resource Body: Food+Shelter+Success, SF State's Basic Needs Initiative, supports students’ ability to reach their full potential at SF State by providing food, housing and financial crisis support programs, services, and referrals. Food+Shelter+Success stands in solidarity with everyone fighting for equality. Equitable access to housing, food, medical care, and education are critical, undeniable, and foundational elements of racial and social justice.
Cosine similarity score: 0.634739026013513
------
------
Resource Title: Health Promotion & Wellness
Resource Link: https://wellness.sfsu.edu/
Resource Body: Health Promotion & Wellness is a unit within Student Affairs & Enrollment Management that provides health education and actively promotes a culture of wellness. We create opportunities for students to enact healthy behaviors through advocacy, campaigns, programming, events, workshops, and peer health leade

## Chat bot goals

In [35]:
# !pip install pyyaml
# !pip install jinja2
# !pip install openai

In [36]:
import yaml
from jinja2 import Template

class PromptLoader:
    def __init__(self, yaml_file_path, context=None):
        """
        Initializes the PromptLoader class.
        :param yaml_file_path: Path to the YAML file.
        :param context: Dictionary of context values to format the prompts.
        """
        self.yaml_file_path = yaml_file_path
        self.context = context if context else {}
        self.user_prompt = ""
        self.system_prompt = ""
        self.load_prompts()

    def load_prompts(self):
        try:
            with open(self.yaml_file_path, 'r') as file:
                data = yaml.safe_load(file)
                
                # Use Jinja2 Template for rendering
                user_prompt_template = Template(data.get('user_prompt', ''))
                system_prompt_template = Template(data.get('system_prompt', ''))
                
                # Render templates with the provided context
                self.user_prompt = user_prompt_template.render(**self.context)
                self.system_prompt = system_prompt_template.render(**self.context)
        except FileNotFoundError:
            print(f"Error: The file {self.yaml_file_path} was not found.")
        except yaml.YAMLError as exc:
            print(f"Error parsing YAML file: {exc}")
        except Exception as exc:
            print(f"Error rendering templates: {exc}")

    def get_user_prompt(self):
        return self.user_prompt

    def get_system_prompt(self):
        return self.system_prompt

In [1]:
ext_json = { 'goal': '[Original goal/problem]', 
            'subtasks': [ { 'subtask': '[Description of subtask 1]', 
                           'importance': '[Explanation of why this subtask is necessary]', 
                           'focus': '[What the student should concentrate on for this subtask]' }, 
                           ... ] 
                           }

In [2]:
user_query = "I want to develop a personal budget management plan that helps me track expenses, save for future goals, and manage unexpected financial situations."

In [2]:
context = {"user_query": user_query, "ext_json": ext_json}

NameError: name 'ext_json' is not defined

In [40]:
yaml_file_path = "/home/adityadev/chillmate/app/backend/chatbot/goal_prompt.yaml"
prompt = PromptLoader(yaml_file_path, context)

In [41]:
print(prompt.get_user_prompt())

Below is the goal or problem that the student needs help with:
I want to develop a personal budget management plan that helps me track expenses, save for future goals, and manage unexpected financial situations.

Your task is to help the student by creating subtasks based on their query 
Provide each subtask with an explanation of its importance and what the student should focus on. Format the output as a structured JSON object.

Final output format:
{'goal': '[Original goal/problem]', 'subtasks': [{'subtask': '[Description of subtask 1]', 'importance': '[Explanation of why this subtask is necessary]', 'focus': '[What the student should concentrate on for this subtask]'}, Ellipsis]}


In [52]:
import time
from openai import OpenAI
import os
from loguru import logger
import json

In [6]:
def call_llm(prompt, system_msg, get_json, model: str="llama3-70b-8192", provider_to_use: str= "open_ai"):
    if provider_to_use == "open_ai":
        url = "https://api.openai.com/v1"
        key = os.getenv("OPENAI_API_KEY", "")
    elif provider_to_use == "open_router":
        url = "https://openrouter.ai/api/v1"
        key = os.getenv("OPENROUTER_API_KEY", "")

    elif provider_to_use == "groq":
        url = "https://api.groq.com/openai/v1"
        key = ""

    elif provider_to_use == "fireworks":
        url = "https://api.fireworks.ai/inference/v1"
        key = "fw_3ZJyTsD8oosbzkc5BW9wAaqk"

    elif provider_to_use == "deepinfra":
        url = "https://api.deepinfra.com/v1/openai"
        key = ""

    client = OpenAI(
        base_url=url,
        api_key=key,
    )
    # client = Groq(api_key=key)
    logger.debug(f"Model being used: {model}")
    start = time.time()
    completion = client.chat.completions.create(
        extra_headers={},
        model=model,
        response_format={"type": "text" if not get_json else "json_object"},
        temperature=0,
        messages=(
            [
                 {
                    "role": "system",
                    "content": system_msg,
                },
                {"role": "user", "content": prompt},
            ]
            if get_json
            else [ {
                    "role": "system",
                    "content": system_msg,
                },
                {"role": "user", "content": prompt}]
        ),
    )
    end = time.time()
    time_taken = end - start
    return completion.choices[0].message.content.replace("\n", ""), time_taken

In [50]:
raw_res, time_taken = call_llm(prompt.get_user_prompt(), system_msg=prompt.get_system_prompt(), get_json=True, model="accounts/fireworks/models/llama-v3p1-70b-instruct", provider_to_use="fireworks")

2024-11-05 12:39:25.948 | DEBUG    | __main__:call_llm:26 - Model being used: accounts/fireworks/models/llama-v3p1-70b-instruct


In [54]:
llm_res_dict = json.loads(raw_res)

In [55]:
llm_res_dict

{'goal': 'I want to develop a personal budget management plan that helps me track expenses, save for future goals, and manage unexpected financial situations.',
 'subtasks': [{'subtask': 'Identify and categorize income sources',
   'importance': 'Understanding where your money comes from is crucial for creating an accurate budget. This subtask helps you recognize the different sources of income, such as salary, investments, or freelance work, and categorize them accordingly.',
   'focus': 'Make a list of all your income sources, including the frequency and amount of each. Consider using the 50/30/20 rule as a guideline to allocate your income into needs, wants, and savings.'},
  {'subtask': 'Track and categorize expenses',
   'importance': 'Monitoring your expenses helps you understand where your money is going and identify areas for improvement. This subtask enables you to categorize your expenses into needs, wants, and debt repayment.',
   'focus': 'For one month, write down every si

In [7]:
from generate_goals import GenerateGoal

In [8]:
user_query = "I want to develop a personal budget management plan that helps me track expenses, save for future goals, and manage unexpected financial situations."
context = {"user_query": user_query, "ext_json": ext_json}
# yaml_file_path = "./goal_prompt.yaml"
model_name = "accounts/fireworks/models/llama-v3p1-70b-instruct"
provider_name = "fireworks"
GenerateGoal(model_name=model_name, provider_name=provider_name).generate(context)

2024-11-05 19:46:33.566 | DEBUG    | generate_goals:call_llm:88 - Model being used: accounts/fireworks/models/llama-v3p1-70b-instruct


{'goal': 'I want to develop a personal budget management plan that helps me track expenses, save for future goals, and manage unexpected financial situations.',
 'subtasks': [{'subtask': 'Identify and categorize income sources',
   'importance': 'Understanding where your money comes from is crucial for creating an accurate budget. This subtask helps you recognize the different sources of income, such as salary, investments, or freelance work.',
   'focus': 'Make a list of all your income sources, including the frequency and amount of each. Consider using the 50/30/20 rule as a guideline to allocate your income into needs, wants, and savings.'},
  {'subtask': 'Track and categorize expenses',
   'importance': 'Monitoring your expenses helps you understand where your money is going and identify areas for improvement. This subtask enables you to create a clear picture of your spending habits.',
   'focus': 'For one month, write down every single transaction, including small purchases like 